In [1]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using MKLSparse
using LinearAlgebra
using Flux, DiffEqFlux
using Optim
using Plots
using DifferentialEquations: Tsit5, DP5
using Random
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

false

In [2]:
Random.seed!(3)

TaskLocalRNG()

$$H = \frac{\Omega(t)}{2}\left[e^{i\phi(t)} (\sigma_+^1 + \sigma_+^2) +  e^{-i\phi(t)} (\sigma_-^1+\sigma_-^2)\right] - \Delta(t) (n_1 + n_2) + V n_1n_2$$

$$H = \frac{\Omega(t)}{2}\left[e^{i\phi(t)} \sigma_+  +  e^{-i\phi(t)} \sigma_-\right] - \Delta(t) \sigma_z + V n_1n_2$$

In [3]:
V = 2π*3.0 # MHz

18.84955592153876

In [4]:
bs = NLevelBasis(3)
bsys = bs⊗bs

id = identityoperator(bs)

sp1 = transition(bs, 3, 2)⊗id
sm1 = transition(bs, 2, 3)⊗id

sp2 = id⊗transition(bs, 3, 2)
sm2 = id⊗transition(bs, 2, 3)

n1 = transition(bs, 3, 3)⊗id
n2 = id⊗transition(bs, 3, 3);

In [5]:
H = Hamiltonian(V*(n1*n2))
H += (n1 + n2)
H += (sp1 + sp2 + sm1 + sm2)
H += 1.0im*(sp1 + sp2 - sm1 - sm2);

In [6]:
states= [nlevelstate(bs, 1)⊗nlevelstate(bs, 1),
         nlevelstate(bs, 1)⊗nlevelstate(bs, 2),
         nlevelstate(bs, 2)⊗nlevelstate(bs, 1),
         nlevelstate(bs, 2)⊗nlevelstate(bs, 2)]

trans = UnitaryTransform(states, [[1.0 0 0 0 ];[0 1.0 0 0 ];[0 0 1.0 0 ]; [0 0 0 -1.0]]);

In [7]:
n_neurons = 16
ann = FastChain(FastDense(1, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh), 
                FastDense(n_neurons, n_neurons, tanh), 
                FastDense(n_neurons, 3))
θ = initial_params(ann)     
n_params = length(θ);

θ = 2.0*rand(n_params).-1.0

627-element Vector{Float64}:
 -0.06937228086515113
  0.3671092755277512
 -0.5345324029179985
 -0.3002323753493943
  0.8349564422875935
  0.4336514600185357
  0.9858209653291647
 -0.25215307580705115
 -0.7505146032696866
  0.4068134194972677
  0.512436109189913
 -0.30952674044677453
  0.4820782600626925
  ⋮
  0.4047637201689982
 -0.004109531533656341
 -0.27818154812574725
  0.9898024346265548
  0.2872689964543662
  0.5567424926736402
 -0.9091575446683373
  0.9506122157764831
  0.029215629683503996
  0.9224104179891424
 -0.09188062915410611
 -0.7089143086398042

In [8]:
t0, t1 = 0.0, 1.0
coeffs(p) = t->let vals = ann([t], p)
                [-vals[1], abs(vals[2])*cos(vals[3]), abs(vals[2])*sin(vals[3])]
               end    
dr = ParametricDrive(Vector{Float64}(θ), coeffs, (t0, t1))
cost = CostFunction((x, y) -> x'*y,
                     p->2e-12*(ann([t0], p)[2]^2+ ann([t1], p)[2]^2))

prob = QOCProblem(H, dr, trans, cost);

In [11]:
sol = QuantumOptimalControl.optimize(prob, ADAM(0.1), alg=Tsit5(), 
                                     n_iter=300, abstol=1e-4, reltol=1e-4)

Progress: 100%|█████████████████████████████████████████| Time: 0:03:42
  cost:  0.6217341168298622


Solution{Float64}([1.0880021223590952, 1.1623299163052243, -1.6035728702050007, -1.1549426809286087, 1.7468975892608665, 1.6843018185361252, 2.397591468843422, 0.48357258248166946, -0.009271698150310247, 0.32695034749055246  …  -1.4707276256326804, 0.2686653640305373, 0.2986541401767957, 1.7524099662131474, 0.25169527932365765, 0.32421568468889567, -3.141104171252357, -2.4410857651368723, 0.474655903424609, -0.7089142624407778], [0.623412885842164, 0.6494202901803994, -0.3448910656642461, 0.5418798578385007, 0.5105017701783175, 0.50579638112434, 0.549428388458658, -0.29056586288533814, 0.5061750732897914, 0.5052694854606281  …  0.6212621583913123, 0.6212495225094583, 0.621794090245396, 0.6218957000782726, 0.621572217748701, 0.6216023299622288, 0.621892675087282, 0.6219491397674137, 0.6217628556592237, 0.6217341168298622])